In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# caminhos (ajuste conforme seu repositório)
base = Path(r"C:/Users/Valentine/Artigo_Mapitos/data_clean/07_Codificado")
out = Path(r"C:/Users/Valentine/Artigo_Mapitos/data_clean/08_Mapeamento")
out.mkdir(parents=True, exist_ok=True)

# arquivos linha-a-linha
file_sust = base / "07_Sustentaveis_Degradacao_Linha.csv"
file_conv = base / "07_Convencionais_Degradacao_Linha.csv"

# ---- FUNÇÃO QUE AGREGUE PREVALÊNCIAS ----
def gerar_prevalencias(file_path, prefix_pratica="pratica_"):
    df = pd.read_csv(file_path, encoding='utf-8')
    # garanta nomes
    df.columns = df.columns.str.strip()
    # coluna de codigo IBGE — ajuste se tiver outro nome
    if 'codigo_final' in df.columns:
        df['codigo_ibge'] = df['codigo_final'].astype(int)
    elif 'codigo' in df.columns:
        df['codigo_ibge'] = df['codigo'].astype(int)
    elif 'cod' in df.columns:
        df['codigo_ibge'] = df['cod'].astype(int)
    else:
        # se não tiver código, tentar colunar municipio
        if 'municipio_ibge' not in df.columns:
            raise ValueError("Não encontrou coluna de código IBGE ou municipio_ibge.")
        df['codigo_ibge'] = df['municipio_ibge']  # cuidado: string, bom ter code
    
    # identificar colunas dummy de prática (se existirem)
    pratica_cols = [c for c in df.columns if c.startswith(prefix_pratica)]
    
    # se não existirem dummies, tentar extrair da coluna textual (p.ex. 'uso_de_praticas_sustentaveis_')
    if not pratica_cols:
        # coluna que contém o texto das práticas
        text_col = [c for c in df.columns if 'pratica' in c or 'uso_de_praticas' in c]
        if not text_col:
            raise ValueError("Não encontrou colunas de práticas.")
        text_col = text_col[0]
        # padronizar texto e explodir em dummies
        df[text_col] = df[text_col].fillna('').astype(str)
        # split por vírgula e strip
        df['_pr_list'] = df[text_col].str.split(',')
        # expandir e get dummies via explode
        df_expl = df.explode('_pr_list')
        df_expl['_pr_list'] = df_expl['_pr_list'].str.strip().str.lower().replace('', np.nan)
        # criar dummies
        dummies = pd.get_dummies(df_expl['_pr_list'], prefix=prefix_pratica)
        df_expl = pd.concat([df_expl, dummies], axis=1)
        # agregamos de volta por original index
        # somar as dummies por linha original (se tiver múltiplas entradas originalmente)
        dummies_sum = dummies.groupby(df_expl.index).max()  # max garante 1 se presente
        # juntar ao df original
        df = pd.concat([df, dummies_sum], axis=1)
        pratica_cols = list(dummies_sum.columns)

    # agora agregação por municipio
    # n_obs: número de produtores respondentes (contagem de linhas originais por município)
    agg_count = df.groupby('codigo_ibge').size().rename('n_obs')
    # proporção por prática: média dos dummies por município (como proxies de proporção)
    agg_pr = df.groupby('codigo_ibge')[pratica_cols].mean().rename(columns=lambda x: "prop_" + x)
    # média do número de práticas por produtor (soma das dummies por linha e média por município)
    df['n_praticas_linha'] = df[pratica_cols].sum(axis=1)
    mean_n = df.groupby('codigo_ibge')['n_praticas_linha'].mean().rename('mean_n_praticas')
    
    # proporcao any (pelo menos 1 pratica)
    df['any_pr'] = (df['n_praticas_linha'] > 0).astype(int)
    prop_any = df.groupby('codigo_ibge')['any_pr'].mean().rename('prop_any_pratica')
    
    # municipio name (se existir)
    if 'municipio_ibge' in df.columns:
        mun_name = df.drop_duplicates('codigo_ibge').set_index('codigo_ibge')['municipio_ibge']
    else:
        mun_name = pd.Series(index=agg_pr.index, data=np.nan, name='municipio_ibge')
    
    # juntar tudo
    out_df = pd.concat([mun_name, agg_count, agg_pr, prop_any, mean_n], axis=1).reset_index().rename(columns={'index':'codigo_ibge'})
    # ordenar
    out_df = out_df.sort_values('codigo_ibge')
    return out_df

# ---- gerar e salvar ----
preval_sust = gerar_prevalencias(file_sust, prefix_pratica="pratica_")
preval_conv = gerar_prevalencias(file_conv, prefix_pratica="pratica_")

preval_sust.to_csv(out / "prevalencia_praticas_sustentaveis_por_mun.csv", index=False, encoding='utf-8-sig')
preval_conv.to_csv(out / "prevalencia_praticas_convencionais_por_mun.csv", index=False, encoding='utf-8-sig')

print("Arquivos salvos em:", out)


Arquivos salvos em: C:\Users\Valentine\Artigo_Mapitos\data_clean\08_Mapeamento
